<h1>Week 3 - Capstone</h1>

<h2>Import data into a dataframe.</h2>

In [21]:
import pandas as pd

# Put csv of neighborhoods into "neighborhoods" variable
neighborhoods = pd.read_csv("toronto_neighborhoods.csv")

# Check that it worked
print(neighborhoods.head())

# Remove any with "not assigned"
neighborhoods = neighborhoods[neighborhoods.borough != 'Not assigned']

print(neighborhoods.shape)

  postal_code           borough               neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
(103, 3)


<h2>Add coordinates.</h2>

In [22]:
### I couldn't get Geocoder to work so I merged the provided csv with the neighborhood data.

coordinates = pd.read_csv('Geospatial_Coordinates.csv')

# Joins coordinates and neighborhoods dataframes
neighborhoods = coordinates.merge(neighborhoods, on='postal_code')

<h2>Install/import libraries that will be needed later.</h2>

In [23]:
import numpy as np

# To work with JSON files
import json

!conda install -c conda-forge geopy --yes # only needed to run once
from geopy.geocoders import Nominatim

import requests # library to handle requests

# Import and set pandas display options, JSON support
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# matplotlib plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# import kmeans
from sklearn.cluster import KMeans

# install and import folium for mapping
!conda install -c conda-forge folium=0.5.0 --yes # only needed to run once
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



<h2>Set up map & show all neighborhoods.</h2>

In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['borough'], neighborhoods['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
# display map
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h2>Set up Foursquare credentials.</h2>

In [25]:
CLIENT_ID = 'PZ1XLTP2AE3UZCNEFM5ZVGAEIITCYBIWITPAZNTUAWMIF43C' # your Foursquare ID
CLIENT_SECRET = 'LOOLAGGD4WSUF42S4OVATNV1DGI1FERGONOGIQYWCZNE0FLZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: PZ1XLTP2AE3UZCNEFM5ZVGAEIITCYBIWITPAZNTUAWMIF43C
CLIENT_SECRET:LOOLAGGD4WSUF42S4OVATNV1DGI1FERGONOGIQYWCZNE0FLZ


<h2>Set up the URL for Foursquare.</h2>

In [26]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PZ1XLTP2AE3UZCNEFM5ZVGAEIITCYBIWITPAZNTUAWMIF43C&client_secret=LOOLAGGD4WSUF42S4OVATNV1DGI1FERGONOGIQYWCZNE0FLZ&v=20180605&ll=43.706748299999994,-79.5940544&radius=500&limit=100'

<h2>Function to get nearby venues for each neighborhood.</h2>

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h2>Use the function to get the neighborhoods, latitude, and longitude from 'neighborhoods'.</h2>

In [28]:
toronto_venues = getNearbyVenues(names = neighborhoods['neighborhood'],
                                   latitudes = neighborhoods['latitude'],
                                   longitudes = neighborhoods['longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

<h2>Just checking things & summarizing data.</h2>

In [29]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

(2127, 7)
There are 271 unique categories.


<h2>Onehot encoding.</h2>

In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()
toronto_onehot.shape
# group by neighborhood and average the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape

(96, 271)

<h2>Print list of neighborhoods with top 5 common venues in each.</h2>

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----" + hood + "----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1     Coffee Shop  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                   Pharmacy  0.04
3                      Diner  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4        Metro Station  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09

<h2>Function to put everything into a pandas dataframe; in descending order.</h2>

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h2>Create new dataframe and display top 10 venues for each neighborhood.</h2>

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()




,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Ethiopian Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Gym,Sandwich Place,Coffee Shop,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Deli / Bodega
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Indian Restaurant,Restaurant,Sushi Restaurant,Café,Pub


<h2>Start clustering the neighborhoods.</h2>

In [34]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# I made all of my column names lowercase, so making "neighborhood" capitalized to be able to merge with neighborhoods_venues_sorted
toronto_merged.rename(columns = {'neighborhood':'Neighborhood'}, inplace=True)
# merge toronto_merged with neighborhoods_venues_sorted to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# For some reason, there are NaNs, so removing those rows.
toronto_merged = toronto_merged.dropna()

toronto_merged.head() # check data

,postal_code,latitude,longitude,borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge",0.0,Fast Food Restaurant,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek",2.0,Bar,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farm
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",1.0,Bank,Electronics Store,Restaurant,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Women's Store,Discount Store,Distribution Center
3,M1G,43.770992,-79.216917,Scarborough,Woburn,1.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,1.0,Lounge,Gas Station,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Drugstore


<h2>Create a map with the clusters.</h2>

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1], # Had to cast cluster as integer
        fill=True,
        fill_color=rainbow[int(cluster)-1], # Had to cast cluster as integer
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examine clusters</h2>

<h4>"Malvern-Rouge"</h4>

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.806686,0.0,Fast Food Restaurant,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


<h4>"Everything"</h4>

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,43.763573,1.0,Bank,Electronics Store,Restaurant,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Women's Store,Discount Store,Distribution Center
3,43.770992,1.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,43.773136,1.0,Lounge,Gas Station,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Drugstore
5,43.744734,1.0,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
6,43.727929,1.0,Department Store,Coffee Shop,Convenience Store,Hobby Shop,Bus Station,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
7,43.711112,1.0,Bakery,Park,Bus Station,Soccer Field,Metro Station,Bus Line,Ice Cream Shop,Cosmetics Shop,Discount Store,Comfort Food Restaurant
8,43.716316,1.0,Motel,American Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Department Store
9,43.692657,1.0,College Stadium,Skating Rink,Café,General Entertainment,Farm,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Escape Room
10,43.757410,1.0,Indian Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Gaming Cafe,Chinese Restaurant,Women's Store,Dog Run,Dim Sum Restaurant,Diner
11,43.750072,1.0,Middle Eastern Restaurant,Shopping Mall,Smoke Shop,Vietnamese Restaurant,Sandwich Place,Bakery,Auto Garage,Dog Run,Diner,Discount Store


<h4>"Rouge-Union-Highland"</h4>

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.784535,2.0,Bar,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farm


<h4>"Baseball & Dogs"</h4>

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,43.636258,3.0,Baseball Field,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Farm
97,43.724766,3.0,Baseball Field,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Farm


<h4>"Parks & Diners"</h4>

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,43.815252,4.0,Playground,Intersection,Park,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,43.789053,4.0,Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
23,43.752758,4.0,Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store
25,43.753259,4.0,Park,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
44,43.728020,4.0,Bus Line,Park,Swim School,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
48,43.689574,4.0,Lawyer,Trail,Park,Summer Camp,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
50,43.679563,4.0,Park,Playground,Trail,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
64,43.696948,4.0,Jewelry Store,Trail,Park,Sushi Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,43.706876,4.0,Park,Jewelry Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
